In [1]:
import yaml
import sys, os
sys.path.append(os.path.abspath("..")) 
from Model.BusinessGlossaryModel import BusinessGlossaryModel 
from Service.DataHubGlossaryImporter import DataHubGlossaryImporter
import Service.Utils as utils



full_path = "../template_business_glossary.yaml"

if full_path.endswith(".yaml") : 
    
    with open(full_path) as stream:
        data = yaml.safe_load(stream)
        dataset = BusinessGlossaryModel(**data)
        importer  = DataHubGlossaryImporter(dataset)


 

INFO:Service.Utils:Created node: urn:li:glossaryNode:operation
INFO:Service.Utils:Created node: urn:li:glossaryNode:operation.general
INFO:Service.Utils:Created term: urn:li:glossaryTerm:Driver4
INFO:Service.Utils:Upserted group urn:li:corpGroup:Operation_team
INFO:Service.Utils:Upserted group urn:li:corpGroup:BI_TEAM
INFO:Service.Utils:Owner urn:li:corpuser:BI_TEAM already exists, omitting write
INFO:Service.Utils:Owner urn:li:corpuser:Operation_team already exists, omitting write


 Linked column 'columnName2' in dataset 'base_api.test15' to glossary term 'Driver4'


In [2]:
test =  "[mysql].[127.0.0.1,3306].[base_api.test15].[columnName2]" 

technical_dataset= test.split('].[')

test

'[mysql].[127.0.0.1,3306].[base_api.test15].[columnName2]'